# Evaluating a Linear Model
"Regression analysis generates an equation to describe the statistical relationship between one or more predictor variables and the response variable." That's a quote from the documentation from an old statistical package called minitab. It's wordy, but what it's basically saying is that creating the model is just the beginning. The interpretation of the results is the important part. 

We left off on our last tutorial by plotting the line of best fit. So at this point, we know how to instantiate the model, draw a line of best fit, and use our model to make predictions. 

Let's import our model from the lab: 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import seaborn as sns

# Load the diabetes dataset
diabetes = datasets.load_diabetes()
df_x = pd.DataFrame(diabetes.data,
                 columns=diabetes.feature_names)
df_y = pd.DataFrame(diabetes.target,
                 columns=["diabetes"])

# split into training and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, 
                                                   test_size=0.2, 
                                                   random_state=42)

# instantiate the model
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True,
                        n_jobs=4)

# train
fit = model.fit(x_train, y_train)

# make predictions
preds = model.predict(x_test)

## Measuring Error
There are three main error metrics used to evaluate linear regression model; Mean Absolute Error, Root Mean Squared Error and R2 score.

### Mean Absolute Error
Since we have our line of best fit, we can measure the distance between each point and the line to calculate the error (residuals). Because this measurement can be either positive or negative, we take the absolute value of each value so the effects of the signs don't cancel each other out, and sum the values, then take the average. 

![mae](mae.png)

#### Manual Steps for calculating MSE
1. Measure the distance between each point and the line of best fit
2. Take the absolute value of the error at each data point
3. Calculate the mean

This is the easiset measurment to understand intuitively 

In [3]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_test, preds)

In [4]:
MAE

42.79389304196525

The MAE represents the average error between our two values, and measures the magnitued of the error. Because we've taken the absolute value of the error, we are unable to determine the direction. 
- 0 MAE means there is no error
- The upper bound depends on the range of your dataset
- To compare MAE into different models, you can convert this score into a percentage

## RMSE
For mean squared error we sum the squares of the differences instead of the absolute values. And then we take the square root of that result: 
![rmse](rmse.gif)

Since the errors are squared before they are averaged, the RMSE gives a relatively high weight to large errors. Meaning that this is the better measurement to use when you want to penazlize large errors. 

In [10]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, preds) # calculate mean squared error
rmse = np.sqrt(mse) # calcualate root mean squared error

In [11]:
rmse

53.8532569849144

This is the most popular metric because it penalizes large errors, giving a more representative error metric. 

## R2 Score- Coefficient of Determination
This metric is called the **R Squared Goodness of Fit** and determines the percentage of the variance in the dependent variable that can be explained by the independent variables. 

![r2](r2.png)

- R-squared is always between 0 and 100
- 0% represents a model that does not explain any of the variation in the response variable
- 100% represents a model that explains all of the variation in the response variable 
- the larger the R2, the better the regression model fits your observations

In [14]:
from sklearn.metrics import r2_score
r2_score(y_test, preds)

0.452606602161738